# IBM Cloud Hyper Protect Virtual Servers Secure Build and Bring Your Own Image lab

This lab will introduce you to two features of the IBM Cloud Hyper Protect Virtual Servers service:

1. Secure Build
2. Bring Your Own Image (BYOI)

The initial introduction of **_IBM Cloud Hyper Protect Virtual Servers_** (HPVS) offered an Ubuntu image which you could then tailor to your requirements, but if you wanted to replicate your changes onto other HPVS instances, you would have to repeat the custom configurations on each additional HPVS instance you created.  Automation tools like Ansible could help with that process, but wouldn't it be nicer if you could just deploy your own custom-built Docker image, that you only had to configure once, onto an HPVS instance?  That's the convenience that the *BYOI* feature offers. It offers the ability to deploy a digitally signed Docker image into an HPVS instance.

You can deploy Docker images that are digitally signed by you or some other person or organization that you trust. In the case of images you build and sign, the _Secure Build_ feature lets you build, and digitally sign, a Docker image within an HPVS instance running an IBM-provided _secure build server_ that offers the confidentiality, integrity and auditability protections inherent in IBM Cloud Hyper Protect Virtual Servers.  

In this lab you will perform the main use case targeted by these two features working together:

1. You will deploy the secure build server in your own HPVS instance.
2. You will use this secure build server to securely create and digitally sign a Docker image that contains a sample application. For this lab you will use a sample _secure bitcoin wallet application_ provided by IBM for demonstration purposes.
3. You will deploy this Docker image that you've built in a second HPVS instance and then run the sample secure bitcoin wallet application contained within the image.

You will be using an open-source application called _Jupyter Notebook_ to run this lab. Jupyter Notebook allows you to enter commands and see the results of these commands. You are likely reading this within Jupyter Notebook right now.  You will also open a terminal within Jupyter and enter some commands from the terminal. The large majority of the commands will be run from within the Jupyter notebook while a small number of commands will be run from the terminal. Those commands that prompt for user input are run from the terminal, while the vast majority, which don't prompt for user input, are run from within Jupyter Notebook.

## Secure Build overview

The diagram below provides an overview of the major components involved in the secure build process that you will be working with in this lab. An explanation of each numbered annotation is provided below the diagram.

![Secure Build architecture](images/sbsarch.png)

1. The IBM Cloud command line interface (CLI) will be used throughout this lab.  It has already been installed for you in the lab environment.

2. You will be using your IBM Cloud account during the lab.

3. The _secure build_ CLI is a separate CLI used exclusively for the Secure Build process.  You will download this CLI in _Step 5.1_ of this lab and will use it throughout the remainder of the lab.

4. You will be building a container image from an application whose source code is in a GitHub repository. The GitHub repository is specified for you in the configuration file you will build in _Step 5.5_ of this lab.  It is the value of the _GITHUB\_URL_ key given for you in _Step 5.5_. 

5. The container image that you build will be pushed to the _IBM Cloud Container Registry_ (ICCR). You will tailor the configuration file in _Step 5.5_ of this lab in order to specify a _namespace_ in ICCR that you will create in _Step 4.2_ of the lab. 

6. The secure build server runs inside a Hyper Protect Virtual Servers instance in IBM Cloud that you will create in _Step 5.11_ of the lab.

7. Your secure build server contains a snapshot of its state.  You can download this _secure build state_ from the server and use it to rebuild the secure build server if necessary.  Although we won't be using this downloaded state in the lab, you will perform the command to download it in _Step 6.10_ of the lab.

**Note:** It is also possible to save the _secure build state_ to _IBM Cloud Object Storage_, but we will not be doing that in this lab.

Okay, let's get started!!

## Initial login to IBM Cloud via the _ibmcloud_ command line interface (CLI)

### Step 1.1 Open a terminal with the Jupyter Launcher

In the browser tab in which you are accessing the Jupyter Notebook to take this lab, you should see a workspace with two tabs:

    1. The tab you are using right now to read this lab.
    2. The Jupyter launcher tab.

Go to the Jupyter Launcher tab.  Click the *Terminal* tile at the bottom left and the Launcher tab will be replaced with a terminal window that you will use to enter a small number of commands from this lab that require interactive user input.

![Choose Terminal](images/opennewterminaliniks.png)

When you have done that, come back here to continue with the instructions.

---

### Step 1.2 Use the terminal tab to log in to IBM Cloud

Navigate to your terminal tab, and from there, enter the command `ibmcloud login -r YOUR-REGION --sso`, substituting one of the following regions for YOUR-REGION:  `us-east`, `us-south`, `au-syd` or `eu-de`, which are for United States (East), United States (South), Sydney, Australia, and Frankfurt, Germany, respectively.  

For your convenience the proper command for each region is listed. From the four commands listed below, copy the one applicable for your region, and paste it into the terminal tab and run the command:

`ibmcloud login -r us-east --sso`

`ibmcloud login -r us-south --sso`

`ibmcloud login -r au-syd --sso`

`ibmcloud login -r eu-de --sso`

---

### Step 1.3 Follow the prompts to complete the login

Follow the instructions to log in to your IBM Cloud account.  The login process from the terminal should be like this:

You will first be given a *Y/n* prompt to open the URL to get a one-time passcode, reply *n*. The reason to reply *n* is because in your lab environment your server does not have graphics capabilities and cannot open the URL.

In order to overcome this, you must copy the URL given in the command output and paste it into another browser tab or window. The goal is to obtain a temporary one-time code for you.
In order to get this code to use to log in with the CLI, you may need to log in to the IBM Cloud Web UI with your browser. If so, follow any necessary login prompts. You should eventually be given a page with a one-time code.  Click on that code to put it in your clipboard, and paste it at the input prompt back in your terminal tab.

When successful, your terminal window will look like this, but with your account information of course: 

```

API endpoint:      https://cloud.ibm.com
Region:            us-east
User:              silliman@us.ibm.com
Account:           Barry Silliman's Account (1e963a246cc69a44df65e277e14239d5) <-> 1996902
Resource group:    No resource group targeted, use 'ibmcloud target -g RESOURCE_GROUP'
CF API endpoint:
Org:
Space:
```

When that is complete, come back to this tab and continue in this notebook. Leave the terminal tab open, you will need it later.

---

### Step 1.4 Target the default resource group

---

#### Jupyter Notebook tip

The text you are reading now is in a type of Jupyter notebook cell called _markdown_.  It is intended for instructions, and includes text and possibly images.

Commands that you will enter (with the exception of the small number of commands, such as the `ibmcloud login -r YOUR-REGION --sso` above, that are run in the terminal tab) will be in a second type of Jupyter notebook cell called _code_.  At the top of this page you can see a dropdown box that will show you whether or not you are in a markdown cell or a code cell. The below screen snippet shows where the type of cell is indicated:

![Choose Terminal](images/celltypeindication.png)

---

You can see from the output from _Step 1.3_ that no _resource group_ is targeted when you log in. The command in the code cell below will target the _default_ resource group. Resource groups are logical buckets that you can define in order to categorize your IBM Cloud resources as you wish.  We will simply use the _default_ resource group which exists already.

Click on the code cell below that contains the command `ibmcloud target -g default`. You should see _code_ in the dropdown box at the top. 

Now that you have the code cell below highlighted, click the _Run_ button from the menu above—it is the triangle icon—and that command will be run, and its output will be shown.

This will target your _default_ resource group. See the debugging tip below the code cell if this command does not work for you.

In [ ]:
! ibmcloud target -g default

#### Debugging tip

Some IBM Cloud users may not have a _default_ group but instead have a _Default_ group, with an uppercase _D_.  If the above command fails, try using *Default* instead of *default*.  You can do that by just clicking in the code cell above and changing the _d_ to _D_ in default.  Then run the code cell again. If that still doesn't work, ask an instructor for help.

---

### A note about running Jupyter Notebook code cells

Observe that to the left of the code cells are square brackets with empty contents. When you run a code cell, an asterisk (*\**) appears between the square brackets.  (If the command runs fast enough you may not even notice the appearance of the asterisk). When the command finishes, a number appears between the square brackets.  During this lab, some of the commands may take several seconds to run, but no command in this lab should take too long. Each time that you run a code cell, ensure that you wait for that code cell to complete execution before continuing with the lab. If the asterisk remains in between the square brackets for more than thirty seconds, ask an instructor for help—none of the commands in this lab should take that long.

The number shown between the square brackets upon command completion is incremented by one each time you run a code cell.  The numbers are helpful in ensuring that you have run each code cell in the correct order.

---

## Create an ssh key for the lab and add it to your GitHub account

### Step 2.1 Create an ssh key pair in your lab system

As part of this lab, you will need to create a key pair that you will then define in your personal GitHub account settings so that this key pair can be used to clone GitHub repositories.

**Note:** If you are an experienced GitHub user and already have ssh keys defined to your GitHub account, you should still create a new key for this lab and remove it from your account when the lab is over, for security reasons. **What is the security concern?** The lab instructors have access to your lab system in order to be able to help with debugging.  If the lab instructors were malicious, they could copy your ssh key, including the private key portion of the key pair.  They are not malicious—they are well-intentioned and kind. But to be completely secure, you should create a new key per the instructions in this step, and then remove it from your GitHub account when you have completed the lab. (The cleanup section of this lab helps you with the key removal from your GitHub account).

Run the code cell below to create a key pair in your lab workspace.

In [ ]:
! ssh-keygen -t rsa -b 4096 -f '/home/jovyan/.ssh/id_rsa' -N ''

---

### Step 2.2 Copy your public key to the clipboard and add it to your GitHub account

The above command created a key pair (a private part that you keep secret and a public part that can be shared safely with the world).  You need to add the public part to your GitHub account settings, so that this key pair can be used to do a clone of the GitHub repository of the sample secure bitcoin wallet application from within the Secure Build Server container.

Run the code cell below to list the public key, and copy this public key into your clipboard:

In [ ]:
! cat ~/.ssh/id_rsa.pub

Copy all of the output from the above `cat` command into the clipboard, and go to [this link](https://github.com/settings/ssh/new) to add this key to your GitHub account. When you get to that page (you may have to log in to GitHub first), put anything you want in the _Title_ field and paste the above output (which is your public key) into the _Key_ field and then click *Add SSH key*.  

**Note:** if you use a password manager and need to use copy and paste to log in to your GitHub account, go ahead and log in and then copy and paste the above output.

---

### Step 2.3 Add the GitHub public keys to your _known\_hosts_ file

Run the following code cell which gathers the public SSH keys of github.com and puts them in your lab workspace's _known\_hosts_ file. This is done only to eliminate a prompt about trusting the github.com host in the code cell after this one.

In [ ]:
! ssh-keyscan -H github.com >> "${HOME}/.ssh/known_hosts"

---

### Step 2.4 Verify you have added the key to your GitHub account correctly

Run the code cell below to verify that you have correctly added your SSH key pair to your GitHub account.  You should receive a message that says _Hi \<your GitHub username\>! You've successfully authenticated, but GitHub does not provide shell access._  If you received this message, continue in the lab.  If not, ask an instructor for help.  

In [ ]:
! ssh -T git@github.com

---

## Create an IBM Cloud IAM API Key for this lab

### Step 3.1 Create an IAM API Key

**Note:** _IAM_ stands for _Identity and Access Management_. _API_ stands for _Application Programming Interface_.

Run the command in the _code_ cell below in order to create an API key for your IBM Cloud account. This will be used in the lab and gives you permission to store your custom-built Docker image into the IBM Cloud Container Registry.

*Note:* If you are an experienced IBM Cloud user and already have some API keys, create a new one anyway, for usage just for this lab, and delete it when the lab is over. (The cleanup section of the lab will help you to delete it). Do not use it for other purposes. This advice is given in order to mitigate the same lab-specific security concern described in the previous section on creating the SSH key.

If, by chance, you already have an API key named _myapikey_, choose a different name for this lab.  You can do that by double-clicking on _myapikey_ in the code cell below to highlight it and then change its name before you run the code cell.

In [ ]:
! ibmcloud iam api-key-create myapikey -d "API key for SBS tutorial"

---

### Step 3.2 Save the value of the API key

The output of the above code cell will contain an _API Key_ field.  Copy the value of this field and save it someplace safe for the duration of this lab.  (The output containing this should remain within the Jupyter Notebook unless you clear the output, but save it elsewhere as well just in case the output is accidentally cleared). There is no way to retrieve this value later from the _ibmcloud_ CLI.  The _API Key_ field should be treated as a password, as that is essentially what it is.

---

## Lab setup concerning the IBM Cloud Container Registry

### Step 4.1 Set your IBM Cloud Container Registry region

For this lab, we will use the global container registry region.

In [ ]:
! ibmcloud cr region-set global

### Step 4.2 Create a container registry namespace

The code cell below contains a command to add a namespace to the IBM Cloud Container Registry that will be owned by you. You can try to run the command as shown, but it will definitely fail—go ahead and try it, it won't hurt anything.  It will tell you that the _secureimages_ namespace already exists.  Change _secureimages_ to something unique by adding your initials and maybe a number or some other unique string to the namespace.  Keep trying the below code cell, changing what was originally _secureimages_ to a different name, until you get a message that indicates success.  For example, the author appended his initials, i.e., he used _bjs-secureimages_ and that worked for him.

**Note:** If you have used IBM Cloud Container Registry previously, you may already have a namespace. Feel free to either use that or create a new one for this lab. There are no security concerns for using one of your existing namespaces. If you run the below code cell with a namespace you already own, the output message will indicate that you already own it. Again, that's fine if you want to use a namespace you already own.

**Jupyter Notebook tip:** If you click the *Run* button above after you've highlighted the code cell below (or any cell for that matter) it will attempt to run the code and then move to the next cell.  If you need to retry because the name you chose was already taken, just click in the cell again and try again.

In [ ]:
! ibmcloud cr namespace-add secureimages

---

## Create your Secure Build Server HPVS instance

### Step 5.1 Download the Secure Build command line interface (CLI):

In order to set up your own Secure Build Server within an HPVS instance, you will use a command line interface (CLI) that is provided by IBM.  This CLI is implemented in Python and is publicly available (including source code) in a GitHub repository, which you will download now into your lab environment when you run the following code cell:

In [ ]:
! git clone git@github.com:ibm-hyper-protect/secure-build-cli.git

---

### Step 5.2 Change the working directory for the remainder of the lab

Up to now, your notebook has been running from the _/home/jovyan/labdir_ directory. For the remainder of the lab, we want to work from the directory that was just created by the _git clone_ command from the prior code cell.  Our new desired working directory is a subdirectory of our current working directory.

Run the next code cell in order to change directories to the _secure-build-cli_ directory that was created by the `git clone` command that you just ran.

In [ ]:
%cd secure-build-cli

---

### Step 5.3 Install Python packages required by the Secure Build CLI

There are some python packages required by the Secure Build CLI, and running the following code cell will install these packages using *pip3*, the Python 3.x-based package installer:

In [ ]:
! pip3 install -r requirements.txt

---

### Step 5.4 Create the Secure Build Server registration file

Part of the "BYOI" workflow involves creating a _secure registration file_, which contains secrets and metadata that can only be decrypted by the running image.  When you build an image later on in the lab, you will download a secure registration file for the image you will have built, and you will pass this file as an argument to the _ibmcloud hpvs_ command that you will use to create an HPVS instance built from your new image. 

The Secure Build Server itself is deployed securely within an HPVS instance with the help of its own secure registration file.  The code cell below will place this file into your lab environment workspace with the name `secure_build.asc`.

This file is posted publicly on the public documentation page for IBM Cloud Hyper Protect Virtual Servers, but the Secure Build CLI repository on GitHub does not contain it.  That's why you are creating it now.  This file is encrypted, and only the Secure Build Server can decrypt it.  IBM Cloud administrators cannot access your secure build server at all, and you can only access it through the Secure Build CLI, which limits your access to specific functions. Therefore, by design, the decrypted contents of this file cannot be obtained by any human.

**Note:** The code cell below will not produce any output, because it copies the encrypted message into a file.  But you know the command ended if a sequence number is placed within the square brackets to the left of the code cell. 

In [ ]:
%%bash
cat <<EOF > secure_build.asc
-----BEGIN PGP MESSAGE-----
Comment: https://gopenpgp.org
Version: GopenPGP 2.5.0

wcFMAxpkEC0f/4+uAQ/9GcamI6ST3+6jXg2QrLXqzBD/P/U4ob5kcAQ4ZWnSP9mQ
aAnebaejgxX6uAuWJbwHJD0kxJKxbzGXzwtoFfztQNkXZxhvpj80/O1EH8sOSv4z
fgABYqVAKpS3Zl4n1YUip0/y92jHlcrsrUkcWhdqYqBv9+uS0w9c1uVmuImscLDR
e2hz9PSUxNAPeqbrhXUlbiVDfu9uAfFhetYhq0di/d12HLqLPNjNX6BmHfU10NuI
XS316DF8OAhLkgHvwTIsGjIGM45BjCESIFPdfYsn3FQl6Ntcjad11pu2XI0o9kFQ
x5oIgTBp5YjxWIpRN5qcHv9kImyxFt03u1XUeqrBLe+A0xfsP9E88fZmL/W5pCeA
eJaHjzTkA3mjtkLVNYXluTc7AB0AxW0BpjP9rpM5IDfe6hzaRuKKBjCDlbiuPPDC
yvuGdr9Jf1MpJ170474yvH5dvmp0Iv4m8qH334mx2PgG2GZS4mdKfloMEPmJ8IIX
5bWR/jIRVqlTZRqL514/C1O9XfmEL/lFvpV47dJxfuziMRw3NF9DUWT4d7SrKGzJ
L4XJZXisIxYCmYXzjcdePg63oAF8vpADjgC/VcN7Ed993mBIkIBmDyQBviYQV/GZ
xwshm611yG+7O235uHSStC+ThtX8yjOUrn4JwVbHy4mc3xdoDrUJ6aGoXNsUehnS
7QFYxZ2daTqFpK/y0lvBYYFAHD8tkeVr2QMSGSUa0vPjf62xT9oTyawcvcE0ouho
WHUAMjBPX7c3HYbzW+vILyJ0x9kmpD7uIUQ9ebCiAhguHagVB4fsb9EzS4qxRsdm
tbmfCpcieXNgRP/gNbKDwCc6O0B1s/5HJo2lrwmbZAN6BGSe8e5p/RggssBXdhKs
Y1d1x8e3eF6fD13/+x2oF5EKv5ddSBbCI2wTFEzAE9butMbF0LpL8MUT7yrUWKsi
zGt94Njza4nD590ipzV3cMY7Rt3oKSg8YBj9ZIPCTrlH3H+PLYFsPV4ho2hhaA6Q
5ZTNdnn+s4XBQDhr11m2NJml3QXxBo4RwiXsaUVUnofbemJNI6/7orf/cG/ecJkT
03Iolz6kCo9uDizSxcbDmvMI7sW/t+0XbFyS1bJiYITigXd3hsfJXcvLcSca8Zy2
fRS0VVT2F2RRTcCpGIwUDhxe5kYCgPX6NSD3L+p5+oZbcsmRRIwwZ8ZeQcHfZcm7
NYe3MGtXR6qKuHRzLl63E1dZ1psw4VhMV+9H1AYV+ATNVh+0Kq0415iEie3i1WMp
i8ChcdJkVg86MWhL1RjmzGq+nUkozeATpFFpscVydMYBa7gm9dE4+oFVXlJS8bFm
EMP7yeHCxEr5YjXb+KEDscibSK1A0AcuRstVYK0Nz/R60uLkiwJBG6mmC8eTvGfF
FaZUA8XBeZVDqCs5mlddsibHG/VgDGNwIIP66djllSPeaeHAjy6smhXh6fKtOg3O
Jl2cKruER3HyUbC0KFqp+AGWNNtIt1+Oc2/0KiuNgURLgmp7jmQABN2zKz1ylyVf
yIhiG5BCA7E+ybde81GjXRWA3Nyok5es6wyhqXWUq+DWodFFciBPAk6PJYhFLyJL
mao+LyCkPXoB/TbPzv3wzcDet9cX1qXReYPOEyEoNRPYTuv1KFtDxRulQErDs2Dt
D1P/oiH/Qba0oYag1PBog/oT5qKT2NcNaRHbARLZWGZKX70AO1c6igFMLEbjd7de
A4+4w7hSpvN+p2TrJVZII+9VsEPkKgbHMQ3kyEJI9R+AKX3FOnWhvnq/+bTos1CB
KZvnQxVBU+IqVhuSDW1wvQS7BIly1WdCkvKkwSfmdEMOFLpTsC7PKGJ1fvWe5B6g
r0jQDe9r1qV6lQLTAwlbbqfyNy19nfeaMdFpUUcj1nIc68uD0brA677aKRhuA5bJ
BF2VFdEC8aGxH5ijF0z+N+NafTbov3tWKNfL0K1HgNG0m6yb7rTAjI49J8L6A1us
kQtW4j8cS4CMMazlGW0OSz+S+HfxGwK8huCjs4afc423BoKLpQGaH5l51QYlfe3I
xFBQX8baktzH1HZEEZ9Gtvkgt0RQ5GKaWhwxeE5fAm7OPPbAO13c451BhAwYHqC+
H7PBhN+hinUGR14ma+YHtVEKmx1MUseqfyI2R5UspCnFb/wEi2jUUd58PR8hMFfU
+FBlChRu6OseNLa/B4m/OHoHitqTGl5FgoySPhzov7lLe3bJqM9lx1OluuJdUHht
0PA14gtpX+6fYIYc5MSjhOJOimrC9AwnWWbxiCVHdwNw9YcRDS6Hm6Efij5yC470
wIFNccuCst/qlCXX9cLSuHs4s32VS322v/KkVpGUkRs65fbmVCFTO621YmhPoQc/
mfghcPNnAJ3xsKbH+jNZ0V+aqOw1euVYMoQVir1fl9vzTZax/Uh7tik1b4xIKJC4
hpM3mciMSUqua+Ty5icg5aDP3z0V8ZVJ740GckPmP4BnGGlXrAMd/8bECiI/uTk+
rfv2+wPuFmnPDPwe2dawg0WVO3cWTeqpsBx2LH5HpGI8M8XP8179q69r0iMYLeWA
NTFnagmHgLrr2z2QKLondFysfNrlSBHKcGVasMaSPTk0NSvOu/DVOhAr0MQA2BBr
xb+SsDxDzYeu5Dx7tqUatL7p6TSugDkVF3Bjev7PfJwHe5FoG9FqDwZmyhZkjaXk
zwchOXTH7yJdFPAHwsWrOLwR3rLkZbrAAWR75Um5Mh7uLET1TlgQWBsXaktsoOWN
YnlY/UMX4TBmbzRrOByNGljNudBpdoxPQi4LGZH+zbZxffeEiekE7AYTPQRshtLJ
tTi8nWhhkm5QwK8Hd1QtEulXoACONhFyVxFmjBYD8/GykhPt3HUOjxD8REwplRNs
o8/uMh9DDPi1WKLEejbpWgyUW8xkAyqY+FtqkcKhjeGN2sVMDCVCO0GUi86MaTlO
gQivXd+4tu+xeDUIORfrkJW499qWNZrsN55er2Q6AVN8khcQibMpg61vghqUumVv
P0LGJe2rIynHUo+nD/Fh9IoxUY4uMRcxKvQVryS8SZaXGjIQL5Q4G26God+aoQip
zw8zSNJiBMjdSKgCv0V8stZ73raNNa0Non/FH39UVD2YMq/lMqcXuEnDys+s+FpW
f9qDaeVWmBRUagwshmebsKsntyPpL80ozTZj0EBUO3A8X9b8OVLzaJQjgFx4YyCE
OvM0mZYP0sZsUTcyvl2wC21B85VygE03s6d1nPTXM0+2iTdDD+btVTupB6WosY0O
gEQ+y1dbCO+ZRbqe1VQeEpOChHLQziyie91Hb1jsUbDn4SOddLgewcX4V2ue0/1W
3LQS2IYggepoiDYFWmo1FLsnoC/hn93T0K9pDy1rfVVCWHdnBEmjhl6JZx9D1bJJ
hIeMcfkIBoHf6NloNjhBJtW0xq9D1qs+ecW05HYU3S+iYzvATDrti5ZMjphkjnQ8
iECojga4NZnh7mXFmS0hOPr+iolObisTqqlguwy5Xv54aeKXNxAu2SyqdD1RVqz8
KcJhQdTXKWMkr3lFAr7Z2DCMKUarHqdjh4fVLwEUrFH0Es5tq55A1sAW2JEOrYg0
YTKQ+RF8Z8U0Xq1ScH6jOkFXy3CVncvU48zTo7onkLXcHRLrMgclRK01+tmk8qCd
lP8OBmPu/zIJgwGL4nQT0U6vgNcWPrjooOH6Ac0FgtKXTSv4FfD6oQqzRcIuycT0
kgN1Le//Q9AXhFGrqukkpGbdzM1KZXkYN1UVxSK0AF4NzK7u35yVmKUVYme2VhgB
HtfiGHjKg71U/syfEKxsKNFBepByqFV2TTJlRZKrtpldIWZSs97wObV3wfW8PjGB
wPwE6cXvXl252TAeUi7cktwjXKdzPKrVimkk7QDkedNgWAm7ut1QQFpyVuQqUx7y
6MxXPnxKeVOScmD9iMfslIZmMLjHyd479+v143fhyIirxl+i8sPbQxk8QzU/Q9sW
MUQs1GLt83GKM5lm6cL9B1E2tRAuqyZ3cuyeZAD2Nq/krUFiO9oU3in7pgUHk+F+
fq6zeftm1JZJifZXw1FYeNEf0fE80j0FvuD5ZQsDCv3Vs/FH+j1noM2uw9PfRCgO
YpI8H6KjgLPx2ZEOJ4ylxJmKkRzmVFo50SIklZtIRM9FQ/vFi1j5rWeN2NIKpyhl
1OLybhMgDpxg1O/SGtLV52AALLFRwFm/fM4fGJCTFChOZVhreVcS7Lpw3cR1C9un
yK9UayzDXjD+AejHbnyrbDpqww95Z1vXvvnXg7vJW3l8PhS9ZQcanpHbvwWqljgN
MEiwX0sGxacs75tHnB8BOYOaTQPazFkAARkoxBsKaJ/4+x6S0HYriE101LBJsoo6
27kguFzIA08SyZ2mVJ1S5pQFTJ6VeZAEbfNOi6bsIIF+dnmZIlnusW8rXusFNS7W
ajzH1apk7wysQkvt4g678GAIL9B30DICEr+w9KVVRgBHX/1oEP/8oDf+xXndfpze
77yYb1hrZ8a9D6UrORZg7r8b4eq8iPrPqX6g8A+70VV3tEvsh1io0Tj4sAU3wnQw
XFcUnPWiiloJDdMpcU9RsCb8NWmNXkROflmzkWTLMOquFk1ciWQ1kzhcTaD/f2uo
ssAsU7zZFKTBdpQXCQRZt88TblVH1evesTZcDxGnDBG0RhtLI9MA1o82QI3dLtMs
fTQBiIMS5pJmGTxnuNiEEOqUZFbY/zFZ0uLXfY5PqzRX8EA0suWYE3RkOiMsXze2
8JQI8kat6Fw3aiDtz3zcq11NB8Nsp6puDFwe6hVu6gyz1PYd8TQocfS54lQaembL
Od0+aNNWgASObhV9OdxvQIdU0u+izyrbb/bG1G5U59kqEVcDoHtNnAsD+nE/aBNQ
hytA+YN9/hEf0TvVGd7uCuzbMNQhGzoqFu/pL0pilPI9x0UYkjJo1gO8tIdhEu09
rgzHjDxmF9fi3lzlriJVOruHmmcEJFov0FZpvnmU6tjU0rRqk3id5cG2kv0LLwOU
7mpLxdUAZxeuq5kCYTI6vsEZ6uFC6+WPBhwzdtpQOINGJ0NrHqvKJ1BDcycFutAb
GsEutr+6E1CFEah9fPMvhIZLNEp19kYA2cJhZ9dKVFUJopX3Uxeh1pXH7N4B7Qgp
t/NZCxbj1fiY1atxzKRnArVaTaNtNdMZPhEKw13bLiQg3UKJQToLldKiI54EgcNR
Bqblm0K4PsT+RZhzn2bCCv1cS3JgAfYEHjsk9d8GGVVoEsc11xGGE3CtIl9qml7d
jaF9+kffxoq/DGq3LrNCF4hIPZQi0n8FBF+Qm8rB0Ld+wcFPOH891yfFzqXC93eS
wonMSZK3qN+sWDP7coj0dlJ3r41TMyR1orIihCpErWp/fhVv0MfYTNtAoTK92qLs
/Z3HeRKFtALmif4vMzZqpFbtDxS2pKZepJnYC9cMWk0X7XZJcnAsAEM/NwnmDW6F
xxy2fd+pcz6en0zTSLwyG8xKQx7bjbvyoa/V8hN/AnvSEmK61W4Ooqg9hXVVF3pl
iUwBES8PH+JP+2huvVyIg7iJJJfdjIrxkWPL8QYw9kkuwyNmEox6nmEWvrNGmmHv
EMfEAhAUQARMNKXT7Hh65MF1QXPicCsjy4LY05oQsdv6UZqdEGxPs/Aq6cLMDrxy
lJqOsY5bNVAchCkBmiqlcU+6jXfqtoLn42NAP08BZDtmeyYKEtHUgeIza0SZbZOv
5kLxQhoAj/kQe6C6ZUCHEm6npf8AEm2l3WyVSIEnglOo/R2x6BUH0Pe2XIiu6jem
c38oJRt8JJDrUrSy1NRYRGsvWAysO2uvRPT6YRc3/BmTUYJEIadX2zYcDQFxMjHu
nmcfSz/VqKaxl6lk/wvmwAWOdcs/WS5tFMzwwtZACqa0NbrfiSnzuadt+j3Zry+J
R74db9EP0fKz9tNwu5jM8E2HMQZmIOO2cprkP1fpZ133yxkgrj9NTLFxpiysi18e
gRFhXhscaitZI0Z/oZEWFqB0yF1z3YJaAs9Xwsc5ciASwxnvKR6L101F/lkeqXPN
IscDWEpxudanLb+50/hrMYx5yiybZ+wvvbmndMlbVXwr6SOytr25AfJNBbvbop0d
VDRbL5ckRyDv/DcJZ2B+JG5YI73lw2OygSM8R/Qd7EwLwMjX/2osOQZV4e1rzDem
+gFYN5GkporJzHGSYA0J5zGB/zQ+NiqrEkA+i15XpAxPfQBOJPee6kviX+nEmxtS
JGmO1/t9eHy/lozCMmqSVkVTyck/FYI+yUFiNStk+/AAWK7k0VAOiZ/gpqRXomTO
iXat0sllyP/nTT+YE+iCPxRGgDBSEhOACA3cY+8MqWW6UmyPdyvspSkRYaB0JlF5
ehhANpmxZA1QfJRcUndGn9OKGuXNlzFm6oM/d1JPrD4Zi4LyFhZfBMFLkS64eYR6
u03Ly8wmO+PBoq56YDQ+gl88Y5svOTYjOGFaB/BCdtX378XHTheMjrlbbRmneDQY
9pc6U25b9w1MVkR/8azbfEV0gEKXNqCNmFfpkzfI25eZIcb5woiR1RWL1WCoIIEm
2WVlmrWB9dCoYMkHr67zIL2Yap+FejXGSqftUh+Z7Brsu2l+v55WwxoPf8bNyGf7
ynmN/kReTdMX60sUzKdqNB1EhaYNGnenO3Bryn6K5DF93datCiCZimy5mz72+ZqH
j4JV+mDs/Wfi9/DJrRj/NHRPT9e+dpZ1igQAp5PDbVg/irgViNGVPybbM0BJ4zXl
RlLLJoIt5iquDWnQdbq3H6EXlCCutENZ6c6zGJNOWWqQFrP5vCPto8pBPb9o15xK
p4MZqLdVw//F0i6AJbCYafO9JwZawbCrn3fzgCtCTwcYvSRFmW8Q7koRwaaAb8I+
S1ZjsQQJfjNsGJggHcJcsKjPxH5SB93KWSHemRafjJBGDb11zHNk0Hb8VY6yjLoW
xi8Jh3ntlYeSPAQqMMBaxYPaKh73Puds7Ro7RbCvQkW/kJCD5f8r29BUZsmk398U
dXKeuQCOG03s7o0M/YzsLNV/HXx22C1g5KPfzjlvt2/NVfwdEJSHaeacu7JivbMC
QLjxzL+PgiNcuzz7NaJcem6yFBbYj5a2ltFy/RmsUkLFCm7pjEsJnmd6VnQrrswF
RYZVwVebHud8quf8cjlu8k83nXasdCljLInhrLSyA3WqPU/qU1G1zZD/4Qqi+XH4
XDHEzfV7wqC5bBQ/5Ebhf0ir32ICHQgQDxdg64xNpzbVse51TgDKGkSMfDVR2aDQ
rfyUWK93LgWRBfn7Uzj3HnXZ7BTGEHMWsFtizs3SloP/0lL3c0fCyzX+SxOO/JgP
I1uAmfKLWZOxV2g4TpfmkZ5h/duDRaTwkBeD5mOPr1pv33AN5ULCtn+Gw8QLXkRO
6ZNs6mMDuJD1OKmUB3xFC9OxPv1r6bmU/c4PAYorrsT3kTP0sgOVXKhED2D3SPhf
c1gG1KorVOWD5ssBNoL3V6fbbdglRlAwzczEue107XxhShQ+Jnj33w2mZGa1bAeH
Sf90LHv8mBz1w1RIvTWG+J649PJZGAbybzLGrMQw3iegy2MbnYtBpptvbcRNwiQd
nIosJwmQHMMY3fPsQYD3pU300g6qkEAbl3BmdJ9g6HjH42tni1tPLJhsz5lHm4BA
vB+s0x+1roEhUO87W0IX3ZEbc/9jDLshsSE4tMu/a7J0fdufQYKNUe6qamBb3ypZ
g3R28GkiWHib2UavsIpeh09c/TYbigdgM5wgHO3ko4AbYmyFRd6P0uy/VDDPf8jL
057wtIXqZFx1QNhWMXHTzlopDlp05aMkbD4qU/HowMi0eWJpvz62OZS5FgICadcb
1GYerJzqmtF92m1zq9sHv8aDBbAxx5j1iLR+1sLslJoyUBiRfkP9b+xcGqzo+upU
wnPfihBQMdSdjmmaYQ/HKr+FCNTr67RB8YbJDrV+cV6Wuh9WDxTqbtRR+A3I1STT
Usz3K3rl6DRhWqM+ldvYfIiamYSuMNllgq30Dige6OGnvpM5U3dGTsENNEuwVfOW
NwpRIz7IYTVkqQgqhzHkHbAhKuxtYmWcuL/YgmLffWHyPVuro+lwy+3gUpPIGtGJ
8pWbfTuLGLnwus3taD0f4lmbcDBVwVN3ICjjXzCaSE5bT2GImFDrU+gnIiGG/ihn
QXOC9VeLnLsa2qtbmcPO/k1sa4bzscN9d/7feeEF5Y50G4Qtzty2ieb2Ekilg9Wv
X9WlZb7sf+U3rnIpo0+Upi8PcoBLQ8e7x0FrU9rQgEW6DMGvOGjvqEc0bZyvHcZ/
1KfWW0WqF7r2EOWLceECX+kY00BwbM4hJGct0HylPHzug1Dk8DX4nMA1nKq53q+u
F5so36s7j/uzef6JHzS3fHbH3d6WZh+xzFrT7IeZpWNgXcgQL+2bBYNKDZB6vwQy
mjfWvGdZb59kmfWSZuVoWN85zbD3l64LNCLWE1Gw9/lQfgLHtfK5gaAGLwAPOnkj
yNmNCxxu0sEprBJpGYkYKfEJ78BsJ7XsPmGebLxgcr6e9+G06ySasxxacwovbHQR
dKC8sFv+Oh4BYhYj3pg1ESMJmILpyNw5lPbuDzxMcvjtiC8uAn3L5hTth6ShkRdR
T/p6ORndWBMnw3Ysw/DVxUuRAxRBIOEpFJFMm+Kiu1wfYfD6BW0H0hVfK6DcbRwb
+sWTUIQc5611iJm03JuVOzWqopKIer91BfR4aHAf9n/BgsgWri8zoVPQ3X6ZfBoF
lsC3C5sH2nWthHUPeIxUFP0kJ6bm80tBm4Nu+k92KM80Qph63nf1A/hy07L+fX2M
KRGtMfe7PonpSTghDh5fFOrDjRwmTctl8IBr7myzlT0p3t0lYJk3iaa3UdfBNGAd
+8RKE3GZVmi6eDgjeKnaoD/dhKvnz4gL9WjxsNdmVvGJw9q24932IHwSUlFlxAC2
1ceV2zQSOiZJ30jx72GTldN/M0vIwqw2HhJTYNZQQ8oH//Ek4/zHe20CxGkJn+77
uOhViFIBIbukNLEfy1sI1SIATJUTQxIbgy3RYdFms/4uyWfFWJoiCT3ouDPueUQn
AyWknvKo4JIAaEjl+cxbfgMTRG4T2wozjN4FP/v9AN5R5VKhod44+Toclyejq0q1
/QEfHx2CBJq37j4+Ar33UPROqhQ71OWxQLZJf5uUxwQFgMdXZAfF2iUF63PSMb1w
vDTGMnz6R23s1XDtCpns4kxsnyV/ttJcXTloV6kLessgWX934w/NwNxMFaU5BL6r
y6VbYjfOUi2qILwM0JGEiW9vHZI/eRubvQETf9LhVVa5HMW7rI2701YttiL+9768
ALVvZJ3+evYhsg73YTkyjuYW3eFmQ1wGW3UhiqaTvhXHvDkfHwIYe+bp35QXlmd3
dxuqEb98ajH0FliBJaiLsgjY0SVRsEfxpiBNxs3BLHxAvXa9fhTh0zpuWhASxeAV
ain86gyIPLhzYT117OL+VkAafb40DvLLE7WslH0QCQO0Ld4Ttz7B/DqRJa5sYvHd
13Gh8z+pLF32bT30NNpRIz5nWIr8wUcr4bYZbt9H80GZym8HmHZRZX/BBrCUTFKg
iZj9xhFK2tOaJfGQEncMWZXtc4w7EwtajSvmNjOssPBdS2n8G4MFdDhi5ZxS7KlH
0bzh/uNR37MZAf1OA5gd2mTcAZKvoDb9R97a0CrGMI977+c5kBLkZ9V+qiUDMP+S
iuLRicFYfSBX6eW7i/cbdFqRrLPdUED0/xIgSuLn+1qeLWcl+A+tipzNPzxZfcNA
ZiW8hkn3M8fkf1iXGYD8lb3YL+8W9Zd07JqVcYFkPZa5Mnd3z6uJiR/HLwzKhlJH
QOm/BwiuLPUNr937OV/WoXTIhIyw1wPppUYTpkgyauQ/f8+cWryEx84KgzwBahKp
AioCweywZ8D0bLgsBhcT5sFz5VeKBQ6/v0jyorvXLvUw6H4FJ5zr7VpU8HEyvwJl
zO1MZoqEi679WSz92SBJBmvcOjWpc0XPzDqRdRupJ1352ZukuV3cnY0/QEFisDOT
gNKFDuOG250da9xbOIloMKLVYgQb6kKPVNR65rfGJR69Gr2qDBB8XrTm1LLCTxQM
wTu3dJumtmlGjvWsoCPgM+nH00RoN3KgWcZS9b9417IIas1A2dsdSMMG3PXJOPtw
8QKMKBUDJLdg9rEfy4GMs1gt/alDHayquyK0/FsaKuVnHjqfyavKahwxJbsxw9ad
w1FVmjHSA1q2Sg7g/jL+Z6ldMedpFUUmzhcjRH0tkYQMipXK38kfpSmcAo+cHJS1
0x1ypUhyhFGblmnyCJbT5OngYss4XlaouA+7xYe85OhU166RtChK++eeGtF6UXqy
VCjlaOXRjl1ohhctsQdidF+5C628Z89Ni1I79656GS9y+A2Hjk/JuNvDs/AU8fma
hUP1oMEXcSiAWziR2flqzLzFYkESqBLT+lAP5rlKic3C4gEPGPVSClVbBMlX3cvc
dgUBMoDz+WMX6bdEiDj9RHXcTtLbdU7YvM8izEiAOGAysRGtaNkS64HscalZhf/7
T+CZnNNaUvx72Ktrsa2Zv9wjo0LLX71g+E6zIG3MD5pX2kgKOZCZJDyT4lI500dj
LkAJjn3XrcKcmYKz2kpFeKiQbV6C0O+DDag5sCoo99Kx+NLsEy/S1am9kWm0kTlD
kMoywK48r6u5BlXPObkD62+wGtJDH6Avb0CxyRARpfXWqyGf6sLKQvnU693ErChV
g9Grsrs55HpRjJniFgTfXVahlQOsWdhNRvGdE5ZgZH5EjQwB6nO5J+hy1XyulOll
rLg5TgvsnQhTs96Yw6Uw08zFFUtSTeSEg4sXOIOOTKhHlqEJvs2cAJSWYVdSJCkD
2zdGKyWijIgs84iMxWcGeoJte6H8636xaCzCwlopfaStjDEloyBu7x8EV9tE2/i1
zlAIzm9cbbBsZrX47RKyB3P8Td864WBbhRb7hCw9gM4wydUu/rXrrW3Qe41eccQ6
UrGKqO5YiRq+0jIfHZ3YQe3qf2PidSrCXtVeSZOaHXHvxL4b8rkn8u5EpsM9H5wb
k9q42xB9P9WiTUIaPP739D4kdW+nkRW1JOqna3OM/gRWF/E6WNM/dKlRoOK0mxdH
JqZQFUeact0o/kROmraBAb5ZMee/oqWj+REdtoLILXDBbPyba+zqSccD54J3tE4L
TYztXTx7e2zFNk9rlA2MEdA3Yj7xuhq6tun1WxlU3mogV0GWM38LPzoPbcooToQI
WbeH8FSb86WaJcd/3fkMlmgy3KLtEvjg19RTSUjHnkUHZyzIH3uXvrTNW3bg1ob+
1tGYCEv282MgFNZKxz9oHk1dM7vsRMPp9mnY4XA3PO2TPX2ydnrn49oQHDkhGsz7
DcfH1a6LtbDLe3C9mO4xQfdPQ9xWOw6aKCbI0o9m09bTuDdYm8oGmwYV2MD7uuw4
3jPw7enyo7bwK+wnkyNa1/zJkXhoZiQt0yH4s4Bys7imuav3HzCc+NRh0O2YCjin
bl4UExiE/qUFMA/Qs47WmW/oETVg7tW1OFprSqF6WSU82Kj31pca4equ2kpoQACY
9fwP7IOJmlPwwhbcC0dXd+OgTr0HksHrKgIp9vRyJ0qhRfBkN3xODev2ArYuDzlB
vHhKv+XIrlzJdgm4gpcQ7nfdEG3gO/r1+uA657HlBFZD7CyFvMPBroqJ111lFsr7
lBngQ42pEi65HJ7rFVkZcocstKgRS92xkjZLLU3yLowoF1groQxHbMbWcnQ1NCuJ
Zqyzual0D8V7b+tX3DOuE+T22DfZRIR2If14KmsryNkTAVWOrcyaf7sHsRNXXVTK
NXzdew3NLh76nmx4iSKYJPA2UbaF0P1E7KmO4B82XIKgpH7Gl/3qyE7p/3F8vd0/
LXN6NqUJlVfWaXV5QqRUV+dv3rKlFu6v08myjTC6vRYnqFkpI3WlG4st8k+s4H4w
HvpBzpszFGIMQRNAY9VHW+HFjVLW/RMX9/im3DEuwvGSif+2t7ZS1wLS65xLO22c
TUv3xTSNy2olRFFA8XD4rfJrP48OcbZZuPRqK4IvHc+iC9ZNNC/HrzPWNsHqCvgE
rEcbfbq7jW/NXHslHdwKI3t/U/ukFe5/0j9HMQlBUzluC7mpP+MnFyxPn2uSRoNs
ydN7PBxHNHhdLrVkl+S6ZUeeBn4GjDPJZ3/Rl316PmHW3547t5LqtjxxEdkm6m9q
XEQ6yDmu953/IBm/mCQvKKdjWRSqLm8j3z5Ao+c3FK1XEbCI7mEg9kqolx4LScyf
6sQBtHPvdgM1Gymp5x9nFKMhgmjVA6/11yQ01Rj2cfhGV/BnUyf1ipvKYNztzezB
GdJhQgX/QV2in/2MNUgWeyysvhR0WWDwqqN9MHZINRJHooxsfUU5qpGeSQILyN4M
+4egEyOlKbLVHkcnH9/ClJ3iJx+N/RiL5LCNS0blcy86IYs5Z4GuZwzFEKPnfOel
QsAIu0VLFRR67L312PF5wLBwv+Nninp9blKF3ZMDDmQXRNG8KyskdY9o8s+RJ2NY
KqLv/PFlXJa+zhbr6WtKOJujyAJRSyIZ3F3w6PrTbWWQWwET064rCgeL/pMPIcbs
twpjPDjEOrHb8iK/U/EVRiny3/4nV2+e2Zrq91lFZKmlvG0jtic/tisMGYohZLC6
MiJj7cJnB5OPFzxKkWWvVfO7VXxkVHnO5vzMF+nPgKBtIy1BTXhV6HUWE0U4v0Lm
znsb//bj76ypB7j3j0OnDYRaaQXcMSZXGzFpXdffhnBhgLrBlgLMy1OYOi9NLJVD
V/aoZUsl1t5+UpK/WXDTglBzRdjL3htiyAVASMFI7WDqeAVkXGbrQSr5dPtqfLVV
t5y5O7no2il2gLinGJxEBzQTPGNBRg5zls9eHWghRO0HqCa+uawN/rlatZ76HKG6
IFdvH6UBVRGIcz5RJHmfRCAaT7S9G7wnvIGFAShhu3w4S5BLR2JPmlSHuXKsEGQA
S/gcIveLcwTLwtNcgYujXuIiGJUhm1urvM4FjlamOYmo+xpiAIr93hoVPk/+IMmd
jTtRVtXMQpY/sz/r5D7dXHi+eaRwZ58gqzW/VquZDuuL59a09wXSP90UFYc8u40K
3q0QN7pif0lVXDcTs8gDICtuiNbXyZZtMSMTVHZxi2gPhou5duzfG4p9FXooqN/7
P/8K6hMEbpLppobOiDDbokny460Z5nOot2saTD6v3HCM3buJo3qvFoaYICh/GZ/d
+tGHvJU1FyhVo+8JVrwEpsD3FF6/7B4giumQxDLrthzrOmGqMVoh5vGzG7+NO4Jq
o0NiN3FDHCaKhXGVUaYv78tOAGbvY6bfQ9JeX6xnTf7uA7YIiMoTlbdgPincu2Cn
Rrn7+e/AXaNsHTWSCxwxY9A98fNUjT2dpRipdQ4D+lLd5BOBbNxCYEHGAj36xuJi
WguJWD1BSFDiwULLSDiP5/3D4O+JgvNwY5sAAyGIF/Q/m9hd0eUuEaSp6GNlPg+7
5xAl64cA5TmemXe9ApOxNWJxJKhlkQ9+4tbqN5ypPijLDY/xPiCMsMrA7UmF+Ah+
fzCjvnETflWjG2F7kyeVm3ATFEWsiOyJUak3KRWSuRXpotFqRCx84VAjW18nVkub
kQCXBbZaOJJZtgG9u9yM4GYe6jCd9+oYkMVs7GIT9txUC8uTaCH1UmEVW5orhySF
lX1FrdpPdZKh0gyQgiwfJM1pyXO9k7HntbA7QHWDNnTGPAKHkPevfI5u10SjrHk9
VK9EprLeXeIfLQEtNSN3e5fHx8xe3j/6p71jKnZ7kLyFtt2do3Iz3Cn/7TxurlNR
EAJ/AoTskenh/jiYuHzAiDhKIZxftJunZ7lhYPPM+5T5ASe8h5XPQWgyDyEB1PU1
5YmRi7zNxVedV1mVfT88BYlSRnYgla2d2bx7c61l2tRQjfgoEuqya3xzzxbiyGoU
UQUrM2u8/h9vmVCDng2NkkWMd2K1lXj5NGsamKA5b7wD/5u6QGo1MqL36JZ9ZBWk
DsDF2oa81zS2RozPL+Jnrvscr2SzIaMZ2xt2WY3dhaUz4p+3FCNZHeYNAG5RdKMS
VWSXluH4/WM5zRAmMZPGxueRlp7hUQI05AGdzOydm1UuCBBKev3Fm7OmUu6IgQxA
zQwNM/UtdHZFLrlUIA54edBwfoyMs1kglhhdlDkg274fIvz9XxyCggDEMw32G0AL
kZvfg/OQucQHj0m88eifgJYaYOLhx15+5mQM8Q/XciPMO8/UhykytdcLYKitorMS
iG+rsC5nFLxUB1cws2mF5QLAJ9EQ83B6CvCYTmO4Ad0KBJ9Pz75C2HDn91ddGjy4
D+5NG8mKjvcaIVqdQqWtAnS1sQhqCOaNNYR6L4qC1LYMRCs38Y+p25hT9bUBLy1x
KKbECwoiSzYgyfHIi0qmVvMyiSJfWDepRJb55tMCfan51FWfauvGoS/Il87+tel6
0U6Y0FB3JHYk2DKwKDBp1SbUE1DHnqMhRmD/HrCBlB1SOdZ6o0TQI45JtAJGJEq8
XKMLbWz4NRLoM/JxVJUC27N0acNdyQcF4fLdwYyuNLW/vfbHklMjHnz5dAhlh/R9
EGFZIkXT1QYMyroGsWsgaJqFOWgZ3f/oOvErqsDqo2AA+ODMc19Uh3Rzc8DK1LVn
Hudd5heCw8rZf8BHNqkuN0qdoDoIw+oltMsCjj9H3F1PGesrPdBWnAthzDG8nQme
IEn/7pukW4HCCiTtz1FXz2Z4RqwzhaiQzqE/CHK+RwASh95iD+5sdZET/FTSghgI
tO+L3k2aUKiU4gLf8vLFCaaJbhMJA3L0GcIe1+ZHix7xzDjhLEI3DfnCzQvH0qcL
0//2QW/DNsVze4l8wS1ZmJyAUkdGy6p6Zliwuwxn8GNOXydzU8hOBpS/nf8dLMMJ
+MY7DprYfv+dcAPLUc97374c9vEK/z9tLmz64TwwX/z8Vou0k0bWG32zFR+q9PLu
Z6r4g5LCRao4MhuXNoJfTlYN4EyQoz6svnkG3hEuoOO35Lss9gIyfSj0liVymSZC
Lho7Mp0S6D1VxO81wmq5eorspYWa0GT3OHgJ1C1k6afwoZuNo9XwUzY4FO+MuFP0
hrTCLtP+Rn8zAzTZ0btuy+jPZILegXRjmpUK5n0UwlQ+QjitPQPLFShLht/lWdaQ
WrbwiqRMr5vsLnPW5O60RO9fPPMaznGA2ov1wyoMBeEj9a2efi3kdBZgPThGLnoR
hRS1OhXRtOZ7i0UTm9BZZq8qsYYIFkvYOT/PUhWejGHAcEpmkBgyVhIVssBZkGrw
rz3414NaNywdG6mLVW1ylOya5ae4twEE6YnzGGX0fwCthqj5gDVAtLo+D5sg6xZD
EGyWcnfMO9PTJY9khsMjVPaR2Q5Jbz5PuYh+Lzdlj/wQFEp1f82UZOt2Py9PLhP7
za1zaicwE4jOvjY5W83bMN+Wve73hlS2weKQ5YFjLGvkpTUCfO6DcTz7faZLY6J2
CfA7QCqgqxGrK3yq+HVAo+iuPFwf7De0c9UCENSVix57uJAz41fXy/XUCqRhPHuD
qexcXFse9KErNqhBAiaM/lRhWBHB1B1kkXvJ2AL0+l/yV4ZOMT6Aj8uy8W68Sk1c
wiwNJcfI+CId/RP8au8aZM7yxryvqtXJtyDgpL5sOw/2jZH4CD1EjV+cGvyww4ZI
0UTKRB/WxYvjQc8n7x5YKPXBXcRSC0y4IKHUudyksva8dZJZdO/toSefZ09W+feo
wIcuvreGYrDY8Y0FvYsILWk+KawDUVc2Ysku7hQRk/TBwiOHOpSKCCyD3m6erVe0
oGS/2S9DPseUuO89bQwzTu3jWb9cD8VEX2q3vWIkwovIH0FF+Y1QVB4iK9BUk13m
g1NeKQR4A3RmhilzPei4UdT7dOHgYSsHMxpsc54tg/CCHNFPmbjy0hHMsViqeGD5
9cJ5+sqhGz+l/sNUQw9HAmOsG9fKfG4TX9dOxJZmZACC9Ys/vaToDuoxsQ4YpSMz
WvOVIDcOpc/ukMCppFpImznZlK73VvepDFjQfb778n7BbsvS8Jy/WS6P2jU7RFJD
Ycbx8EkjVOr2jJNYHB81fbtn/9klyW/DxaXgz69dKwWMPv6oWyuR1+FzwRz84Yza
4amam2y1XgXTKbf6CbJ4FSOn2VNS63Zo3bvGsyI3iKZ07pvZ2rOgl9dTwEcKLChu
8h8rM3bKjHaQkX4ReWXiLvxkPJ7s6nK/h9ml5AO+KRIktk9pVI6hQJzgOIbBDOTG
8jPpZUEiBX/3lPdB8R/9tvFpUb8NCMiVy0WJuwDDvm9jqJA3BR25zDNaZHp+GDyv
xu2qo+Qsy51Oa+2A2tju8obfbRbm6tygfDYPmUt/wcfZPNBhbRD2GndPrIuEtCa8
0fktPwk0+GQEDJlzRY00W/Tm1omk72mWF1yMQ6AcBfaZeo51zBIBXfbJmlz2hf2j
W9ENjHW7pud55Q6q7AKny/4XMhtHg9CUN50zerBC4afP5rmqxfj3TP+U9J7UKeol
O/6t4APWCTpRBKNSlO7kNGbzo68hKWyg4hbJZ7/fQzbd3wc4Hy4SkHcqLby3zwu0
FGaA6UAyGo+n7j9xNodX6VfegbQugz3ZzEmvVTPHdesnVIOR+q/823gCX2FPM580
3F1QzWWh3bqixGlIcXW1oh0/aqggXvuYgXB8IX1TSVnagNgsPgj6y1O1V4NyhpQ4
JSYM1a4FUv60vVZOofej1TdVgUj3IpFQXgjMAVYIBwFChustrwUvPr2S7IaHUQuG
TbLbo1Bz2pmsip4TXLR1poX5VN4d1Y9VAlySyhadsq2cjpExUdI2GcvPx4PhYnhT
WmjcqI/pcZLXYxdM2xH3gRWv+fgw7HjpVPePZji4Q5RCPIg0jOUoCMM7qaKoITk+
bH3rs7I2l5K39kuhyolBOznYMxKdQmkREzqay5Bla+6IEgD27mx/C59RrrI6UeEt
Xw/UtsOG/9ceXsZ17NxEZZkCD0/Fg1O1YpWMYT0qVKB70zB1p10gF3SwfDu31stI
Oo0wo6Fu7rZvV5r8ZaztFGRK+4bkV+tkbeMRxHW4cIzlN/RmhKrjRUvpT1lQOpms
wVP7zErlpcmNKB5uHSUQGpuLDChY8wm4h/QR7+Ib3k70QiHRLD1o6xFC96SFJmZy
yiHp0YqkLcB4ziM62wKMxoUI9sFIRrMQsXus7g4oWliu9zTgsPATRctWgD4Kyiml
zSyqGlzj9g7I+8UJpAllLWiafz5nRQH8Kazic1StpbaheTc6cMBVKyYOeagJtlxi
Bf9GU9UDUUehqq6tbs9boDiIKPQfWilDdC83FewQHdQMVNwAGchUyF5RxtZzmWKa
4Z8WsLQyuRnUn07PEPCk+xEcDZ4wOyFFI7AZFlQO4AoIw3AuprHvAnsG6QtYhlY3
kWKwnGwQoXzlIP+ArtL8lHssnL5g2Lx1Rd/rG0ijP3nJyaCIUyFG1WWyvb3Kwdko
T5Vm4Er8xOUDgqJS8GfobAqIBRWOmb3iIUamX+K6JeJbWZ4/MgLHVBiYZCdOSP9V
OI/SvTN6EjQ5Hnp4QJ3QwynuAiLPIPmsSSWvzFTbU++enZjL7RVD/oH3gsnn+pIw
ajbprvQyASENt8l7/xAFu5QDmK6tMGulz2pADUcmee47ZWJd0g3fPSGIhj7CxREd
lm07XjWJo7gNNCLLA1ikiplvtnFmhn4IngmabVwz9ZVUn+YJnRA4q+Ip0NcfkEmA
Qu/C9j58CnTHiab3LpLBSDWNYstoPoPpmsi910YWedhHbTPEUdw22vprSZI6B6sK
48eeDyaoRiEJGDmeehSDCBMLLY2pCrzz9EZ7ft/G3T1Ci8fpk3PkaiFQUHz9CM+m
29LqzciIVLEffdxnvD4kv+FVNEIeBbY/G3LiiwJhHjOBkR2pKjGSZVGONhpsX/AY
s6K6rT8emCx14+RDVcBIsxGakFvutzhH8EHKlJgzNsTCae0RVETDvJ/KzqcPTS+4
06c3M/PNHiqM2TMNUQVRbWw5WXv0w+89YZqjIVIaFwbAkb4Cw7cQs7oHJSIbVcBt
UY9tFynT7C+cH/wGzLIh1kWolUz9FmOUHLqHaQdErG5pSZMjnwPUPq34XX/xjL4W
ovX4PiytuwoFDDV5+pUpwnrRR4UALfxP6O9If/R2+HZ+qe3cEjK/YqA9Lfl3Bsug
IDoKcaGXpejMAHh7WYbAfWhSSbhHs6mMZPt9up0IydF5SsndHjl3D0KLZ0UEVJn2
Kg65Nvtr03vmmNaGsVwSbUEsJZKjTKNdW/mFVhLfir5UWRvBmdGvNTIQzXFKrtSB
5LvXcwmHrY4S1nwY6jGkrsQQ4B2Q14M4pFl7UWFS7G8ISQOVJLR2o1lQ8C8kvwhE
g7zRjbW0/Hp4YEEdq1vaZABSuCz2m5FreVV0kU6enVa4WK4RJ5MCC6yGZCmsiir5
ODYlhgp63PO5S+pGr87wx1HozNjc1VrnUo9A8NwFpv/yN3qFfvFcD/U8GbsQIu9B
mKRbjvReMyFtXrnKn2vbbvWvTsTBK58aedINdNRpqOr4jQuDFRjgxH443dvuDByh
Nt3cWwyh+CduLVZbJzcxP84j824gc5+1PLpopK83C8JZTB5wC374g+36Tn8e2nYW
UMpRj8uuza6VRaHbUM8bqOlv//y0giGKb14lrXWlkM7bHOQwzvONmr2pvlp/0QDc
VhXWfwcMoe76l8bgote0pDJi7Xf+5rvVqSLFCNQ6HZ3JWkuZsoIIvLIgsIbcOj2T
U1Gg0jW2nPIS/QHC4BrPHUZLtzD+EFKt8GdmkKJHCcoQ3T+1+UrtGHh3lhtVUZda
Fxj++vmk25rJEC6eS1zh9ADrt7PWGBzYvJDflUrHYbkqggASrzYqYpAtyvjkwGCC
/YInZ8eZG9gN81EWkZrP8jRObirL77rzi+sceeiV2PKJBiFUeyf6BQQlwcGGRZIq
Uf3YZ4D9AcWW7UgQiCpkjosrGY1k7evgMZoMoViDRNZGFxZisPEV12qdqpEK0dIc
DkrXQTicJCrpBeWgX5Dj+Xhc7CDhk0W4I837iHZPl5AOG4iQB3rhS3ZRomfQUuXS
j/lLHltrcRnxqOBhmvCVBFucGUwb7kS6Aj+Sm8a50o+Gt9cS/wp6FewDGceYirlZ
FV2IizLUqDxFYIJLYRrV/zmKPymN9A7e5Ex5GbDxpqMg
=c6rC
-----END PGP MESSAGE-----
EOF


---

### Step 5.5 Create your secure build server custom configuration

Anybody who uses the secure build feature of HPVS will use the same Secure Build Server image, but that image is deployed to that person's individual HPVS instance, and that individual instance is uniquely customized through a configuration file. This ensures that your Secure Build Server can only build the application you specify, and push it only to the registry that you specify in the configuration file which is created by the below code cell.  Don't run it yet!  You have to tailor several settings in the code cell below before you run it:

1. For the value of the _DOCKER_REPO_ key, replace `YOUR-CR-NAMESPACE`  with the name of the IBM Container Registry namespace that you either created earlier in *Step 4.2* of the lab, or that you already owned.

2. For *both*:

    a. _DOCKER_PASSWORD_

    b. _DOCKER_RO_PASSWORD_

 replace `YOUR-IAM-API-KEY-VALUE` with the API Key value you created earlier in the lab in *Step 3.1*.

Leave all of the other values unchanged.

Once you have made these substitutions in the below code cell, run the code cell.

**Note:** This code cell also does not produce any output within the Jupyter Notebook for the same reason as the previous code cell.

In [ ]:
%%bash
cat <<EOF > sbs-config.json
{
    "HOSTNAME": "sbs.example.com",
    "RUNTIME_TYPE": "classic",
    "CICD_PUBLIC_IP": "",
    "CICD_PORT": "443",
    "IMAGE_TAG": "1.3.0.11",
    "CONTAINER_NAME": "SBContainer",
    "GITHUB_KEY_FILE": "~/.ssh/id_rsa",
    "GITHUB_URL": "git@github.com:ibm-wsc/secure-bitcoin-wallet.git",
    "GITHUB_BRANCH": "master",
    "IMAGE_TAG_PREFIX": "s390x-v1",
    "REPO_ID": "sbs",
    "DOCKER_REPO": "YOUR-CR-NAMESPACE/secure-bitcoin-wallet",
    "DOCKER_USER": "iamapikey",
    "DOCKER_PASSWORD": "YOUR-IAM-API-KEY-VALUE",
    "DOCKER_RO_USER": "iamapikey",
    "DOCKER_RO_PASSWORD": "YOUR-IAM-API-KEY-VALUE",
    "DOCKER_CONTENT_TRUST_BASE": "False",
    "DOCKER_CONTENT_TRUST_BASE_SERVER": "",
    "DOCKER_BASE_SERVER": "icr.io",
    "DOCKER_PUSH_SERVER": "icr.io",
    "DOCKER_CONTENT_TRUST_PUSH_SERVER": "https://icr.io",
    "ENV_WHITELIST":  ["ZHSM", "APIKEY", "IAM_ENDPOINT"],
    "ARG": {
    }
  }
EOF


---

### Step 5.6 Verify your custom configuration file

You can run the code cell below to display the contents of the file you just created to ensure it looks good.  If you made a typo, you can probably fix it by going back to the above code cell and running it again. 

The code cell below pipes the output of the _cat_ command to a utility called _jq_ (for _JSON query_) which expects well-formed JSON (JavaScript Object Notation) data. If you made a mistake while updating the file that causes a JSON syntax error, the _jq_ utility will detect it and print an error message.  

If you put in a wrong value that doesn't cause a JSON syntax error, _jq_ will not catch that, so inspect the output of the code cell carefully.  All variables that you are required to change started with _YOUR_, so it isn't a bad idea to look for the word YOUR in the output—you shouldn't find it!

In [ ]:
! cat sbs-config.json | jq .

---

### Step 5.7 Generate client certification authority and client certificate

Secure communication between you and your secure build server is achieved by using mutual Transport Layer Security (TLS). TLS is the same security protocol that protects most Web traffic today, but most Web interactions only require the server to be authenticated.  Mutual TLS refers to the situation where not only does the client want to ensure that the server is legitimate, but the server also wants to authenticate the identity of the client.  Your Web browser is constantly receiving server certificates and verifying them while you surf the web. Client certificates are what are used when the server wants to verify the client.  

Create the client certificate and client certification authority (CA) by running the code cell below. You should receive two lines of output, with the second line being `INFO:root:client_certificate: generating client CA and certificate`

In [ ]:
! ./build.py create-client-cert --env sbs-config.json

---

### Step 5.8 Generate server certificate and server key

Create the server certificate and server key by running the code cell below. You should receive four lines of output, with the fourth line being `INFO:root:server_certificate: Successfully generated server certificate`

In [ ]:
! ./build.py create-server-cert --env sbs-config.json

---

### Step 5.9 Display your environment which shows your client certificate and CA and your server certificate and key

The next code cell will display the client certificate, the client certification authority, the server certificate and the server key, all in base64-encoding.

**Note:** Keep the output from the below code cell.  You will be copying most of its output into your clipboard in a subsequent step.

If you already know about _base64-encoding_, go ahead and run the code cell.  If you don't, read the next paragraph if you're curious about what base64-encoding is.  

Base64-encoding is a means of encoding any data, including binary data, into human-readable characters.  It takes groups of 6 bits, converts these six bits into a number from 0 to 63  (2 to the 6th power is 64), and replaces these 6 bits with one of 64 human-readable characters.  Which 64 human-readable characters were chosen?  Lowercase alphabetic characters a-z for 26, uppercase alphabetic characters A-Z for 26 more, the digits 0-9 for 10 more, which gives us 62.  Then `+` and `/` were chosen to make 64!  It should be noted that base64-encoding should not be confused with encryption.  It is easy to convert a base64-encoded string back to its original format.


In [ ]:
! ./build.py instance-env --env sbs-config.json

---

### Step 5.10 List your current HPVS instances

Run the following code cell to list your current HPVS instances. If you are new to HPVS, you won't have any.

This lab uses the free version of HPVS instances, and IBM Cloud users are limited to two free instances at a time, and the lab requires two instances. If you are an experienced HPVS user and you have paid HPVS instances, they won't be affected by this lab.  If you are currently using one free HPVS instance and don't want to delete it, talk to your instructor who can help you work around this situation. If you are currently using both free HPVS instances and can't delete at least one of them, then unfortunately this lab will not work unless you are willing to incur the costs of creating chargeable HPVS instances for this lab.

In [ ]:
! ibmcloud hpvs instances

---

### Step 5.11  Create your Secure Build Server with the _ibmcloud hpvs_ CLI

The code cell below has the first part of the `ibmcloud hpvs instance-create` command filled in properly.

**You need to make two changes.**

**First**, you need to change _YOUR-LOCATION_ to one of the locations as explained in the following paragraphs.

If you logged in to _us-east_, then choose one of these three locations: `wdc04` or `wdc06` or `wdc07`

If you logged in to _us-south_, then choose one of these three locations: `dal10` or `dal12` or `dal13`

If you logged in to _au-syd_, then choose one of these three locations: `syd01` or `syd04` or `syd05`

If you logged in to _eu-de_, then choose one of these three locations: `fra02` or `fra04` or `fra05`

**Second**, you need to add arguments for the `CLIENT_CRT`, `CLIENT_CA`, `SERVER_CRT` and `SERVER_KEY` variables.  All you have to do is copy them from the output from two code cells above, from _Step 5.9_.  Copy the four variables by selecting the output starting from _-e CLIENT_CRT_ and extending to the end of the output. In other words, you are copying almost all of the output from _Step 5.9_ except for the few lines of output that precede the start of the first *-e CLIENT_...* variable that is listed.

I have left a blank line in the code cell below in which you can paste these four excruciatingly long environment variables. Click in the code cell so that your cursor is at the beginning of the blank line 2 of the code cell, and then paste in the variables at that spot.

**Tip:** When trying to copy this long string in the clipboard, when the entire string isn't visible on the screen at once, I have had smoother scrolling when starting from the bottom and then sweeping up to the top to select the text.  Your experience may differ.

Once you have tailored the code cell per these instructions, run it.  This command may take several seconds to run.

In [ ]:
! ibmcloud hpvs instance-create SBContainer free YOUR-LOCATION --rd-path "secure_build.asc" -i 1.3.0.11 \

# copy and paste, in the blank line above me, the super long -e CLIENT_CRT=...  -e CLIENT_CA=...  -e SERVER_CRT=...  -e SERVER_KEY=...  from the output of the command \ 
# a couple cells above.  you can leave me here because I'm just a comment.

---

### Step 5.12 Check the status of your Secure Build Server HPVS instance

It will take several minutes for your instance to be built. The command output from the previous code cell in *Step 5.11* conveniently lists the _ibmcloud hpvs_ command you can use to display the status. Copy the entire `ibmcloud hpvs instance crn:...` command above, which has your unique instance id (shown in the output of the above command, it starts with _crn:_) into the code cell below and run it periodically (about once a minute) until the instance is successfully created. 

**Tip:** Your instance is successfully created when the value of the _Cloud state_ field of the output is _active_. For the first several minutes you will see that the _Cloud state_ is _provisioning_ and that most of the values listed below _Cloud state_ are _not available_.  At a certain point, you will see that those values that were _not available_ have been populated, but you may see that the _Cloud state_ remains as _provisioning_ for a little while. Ensure that you wait for the _Cloud state_ to change to _active_ before proceeding with the lab.

In [ ]:
! 
# copy and paste above the entire 'ibmcloud hpvs instance crn:.....' commmand from the \
# output from the previous command, and run it periodically (every minute or so) until 'Cloud state' is active.

---

## Create your sample Secure Bitcoin Wallet Application BYOI image ##

### Step 6.1 View your custom configuration file again

The code cell below will show the current state of your secure build configuration file.  You have already looked at this file from an earlier code cell, but we're repeating it here so that you can observe the key thing I want you to note from the output—that the IP address value of the _CICD_PUBLIC_IP_ key is blank.

In [ ]:
! cat sbs-config.json | jq .

---

### Step 6.2 Add the public IP address of your secure build server to your custom configuration file

Now that your secure build server instance is running, make a note or copy the IP address shown in the *Public IP address* field from the command output from *Step 5.12*. The code cell below uses an awkward-looking, yet simple _sed_ command to add the IP address in the proper spot.  

**Editing the below code cell is required:** Replace the _xx.xx.xx.xx_ string below with the Public IP address of your secure build container, then run the code cell. Make sure you leave the double-quotes surrounding _xx.xx.xx.xx_ in place. You should not see any output from this code cell:

In [ ]:
! sed -i -e 's/\"CICD_PUBLIC_IP\"\: \"\"/\"CICD_PUBLIC_IP\"\: \"xx.xx.xx.xx\"/' sbs-config.json

In [ ]:
! cat sbs-config.json | jq .

---

### Step 6.3 Update your /etc/hosts files

You need to update the _/etc/hosts_ file in your lab workspace in order to map your secure build server's Public IP to the hostname of `sbs.example.com`. You will want to add a line that looks like this, substituting your instance's Public IP address for `xx.xx.xx.xx` in the below example:

`xx.xx.xx.xx    sbs.example.com`

Run the following command in your terminal tab to get to an editor and update your /etc/hosts file. Ask an instructor for help if necessary if you are not familiar with the _vi_ editor:

`sudo vi /etc/hosts`



---

### Step 6.4 Check the status of your Secure Build Server

Run the following command to check the status of your Secure Build Server instance. The value of the _status_ field in the output should be empty, i.e., `"status": ""`:

In [ ]:
! ./build.py status --env sbs-config.json

---

### Step 6.5 Initialize the configuration for your Secure Build Server instance

Run the following code cell to initialize the configuration for your Secure Build Server instance. The status of the output should be `OK`:

In [ ]:
! ./build.py init --env sbs-config.json

---

### Step 6.6 Start the build of your sample secure bitcoin wallet application image

The code cell below will start the build of your application image. The command returns quickly, as soon as it submits the request to start the build.  Your status message from the command output should say `OK: async build started`, which indicates that the build of your image has been started up on your Secure Build Server container. The command does not wait for the build to complete. The build will take about twenty minutes. Try to check out the next two steps before you take a break.

Run the below code cell to start the your build:

In [ ]:
! ./build.py build --env sbs-config.json

---

### Step 6.7 Check the status of your application image build

Check the status of your build by running the code cell below.  You can check it periodically—your build should take about twenty minutes to complete.  You can run this code cell periodically, as well as the command in the subsequent code cell (_Step 6.9_) which displays the log messages from the build.  For now, check the status by running the code cell below.  Your build has completed successfully once the output of the code cell has the value of *success* in the *status* field:

In [ ]:
! ./build.py status --env sbs-config.json

In the above code cell, until the build completes, you may see several different statuses, including:

_cleaned up_   This status can appear at the beginning of the build but you may not see it as the status quickly changes to _github cloned_.

_github cloned_  This status appears most of the time as this status remains until the image build is complete, and this takes the largest amount of the entire build time.

_image built_  Once the image is built, but before the push of the image into the IBM Cloud Container Registry is complete, you may see this status.

_image pushed_ Once the image has been pushed to the IBM Cloud Container Registry, you may see this status, but usually the build completes very soon afterwards, so you may not see this status, as it should hopefully quickly change to _success_ status.

_success_ Once the secure build process has completed successfully, you will see this status. Once you see this status, you are free to move on (Step 6.10). Remember, you can run the command to display the build log periodically (Step 6.9) while you are waiting for this _success_ status. 

---

### Step 6.8 View the log from your application image build

Your secure image build will produce a lot of output in its log. 

You can check the log of your build by running the below code cell. 

**Jupyter Notebook tip 1:** In some environments this output may be placed in a scrollable window, but in other environments all of the log output may be placed into the notebook, which makes navigating to the rest of the Notebook more difficult.  If this happens to you, you can enable scrolling by clicking in the output, then right-click and choose _Enable Scrolling for Outputs_. This will place the output into a scrollable window. 

**Jupyter Notebook tip 2:** Additionally, you can clear the output of this command from your Jupyter Notebook by clicking within the output, then right-click and choose _Clear Outputs_.  (The build logs are retained on the Secure Build Server and can be retrieved at any time).

**Jupyter Notebook tip 3:** Running a code cell clears the output from any previous execution of the code cell, so if you want to view the log more than once, you don't need to clear the output as described in the tip given in the preceding paragraph.

In [ ]:
! ./build.py log --log build --env sbs-config.json

---

### Step 6.9 Download the state image of your Secure Build Server

Run the code cell below to download the state image, which is a backup of the state of your Secure Build Server instance. This can be used to recreate your Secure Build Server if necessary. This backup also contains the logs of any builds you run on your Secure Build Server.

In [ ]:
! ./build.py get-state-image --env sbs-config.json

---

### Step 6.10 Verify the manifest file from your build

The Secure Build Server creates a signed manifest file for each successful build. The manifest file is useful for audit purposes. You can verify the source and integrity of the build and the built image or you can pass the manifest file to an auditor to do so. Run the below code cell to download and verify the manifest file:

In [ ]:
! ./build.py get-manifest --env sbs-config.json  --verify-manifest

The above command downloads the manifest file and verifies the digital signature on it.  In the last line of output from the above code cell you should see *verify=OK* which indicates that the digital signature is valid and is proof that the manifest file has not been tampered with.

---

## Create HPVS instance using the BYOI image that you just built

### Step 7.1 Download the encrypted registration file for the image that you just built

In order to create an HPVS instance using the application image you just built, you must download an encrypted registration file for your application image that was created by the build. This registration file can only be decrypted, and thus used, by your application container image.

**Copy the command in the code cell below (everything but the leading exclamation point) and _run it in your terminal tab_**. As part of this command, you will be prompted to enter a passphrase three times—the first two times to set the passphrase on a new key, and then this new key is used to sign the registration file, so you are asked to enter the passphrase you just set.  You can set the passphrase to whatever you want.  For this lab, you only need to remember it for the duration of this command—it's not used in any subsequent commands in the lab.

Note the value chosen for the *--key-id* argument:  *secure-build-6d0f0e4-1*. 
If you look at your command output a few code cells above, from _Step 6.8_, where you checked on your build status, you can see this hash at the end of the value of the *image_tag* key, i.e.,  `"image_tag" : "s390x-v1-6d0f0e4"`. 

This value _6d0f0e4 was set by the secure build server. It is not unreasonable for you to wonder why the secure build server chose this particular value. It would be unreasonable for me to fail to provide an answer now that I've piqued your curiosity. That value is the beginning of the hash of the most recent commit of the branch of the GitHub repository that you specified in the secure build server configuration file that you created in *Step 5.5*. Check it out for yourself at [https://github.com/IBM/secure-bitcoin-wallet](https://github.com/IBM/secure-bitcoin-wallet). The secure build server appends this to _s390x-v1-_ in order to form a tag name for the image it has built. 

The *--key-id* value can be anything, but putting this hash value as part of the name helps to keep from confusing different keys if you have multiple keys.

**Note:** This command won't work if run in the code cell because of the need to type in a passphrase.  That is why you have to run it in the _Terminal_ tab.


In [ ]:
! cd /home/jovyan/labdir/secure-build-cli && ./build.py get-config-json --env sbs-config.json --key-id secure-build-6d0f0e4-1

**Jupyter Notebook tip:** If you accidentally ran the above code cell within this notebook instead of copying it into the terminal tab and running it there, it may never finish because it needs user input.  If the square bracket to the left of the code cell has an asterisk, you can interrupt the command  by choosing _Kernel-> Interrupt Kernel_ from the Jupyter Notebook menu above. 

---

### Step 7.2 Launch your BYOI application as an HPVS instance

Make the change described here to the below code cell and then run the code cell to create your HPVS instance using your BYOI image that you just built.  The value for the `-i` argument must be the value of the *image_tag* key in the output of *Step 6.8* which checked your build status.

**Tailoring the below code cell before your run it:** You need to make two changes:

1. You need to change _YOUR-LOCATION_ to one of the locations as explained in the following paragraphs.

    If you logged in to _us-east_, then choose one of these three locations: `wdc04` or `wdc06` or `wdc07`

    If you logged in to _us-south_, then choose one of these three locations: `dal10` or `dal12` or `dal13`

    If you logged in to _au-syd_, then choose one of these three locations: `syd01` or `syd04` or `syd05`

    If you logged in to _eu-de_, then choose one of these three locations: `fra02` or `fra04` or `fra05`
    

2. Ask your instructor for the APIKEY which allows access to the instructor-provided Hyper Protect Crypto Services instance, and replace _YOUR-INSTRUCTOR-PROVIDED-APIKEY_ with the value that the instructor gives you.

In [ ]:
! ibmcloud hpvs instance-create securewallet free YOUR-LOCATION --rd-path sbs.enc -i s390x-v1-6d0f0e4 \
-e "ZHSM"="ep11.us-south.hs-crypto.cloud.ibm.com:8367" \
-e "APIKEY"="YOUR-INSTRUCTOR-PROVIDED-APIKEY"

---

### Step 7.3 Check the status of your BYOI application instance

It will take several minutes for your instance to be built. The command output from the previous code cell in *Step 7.2* conveniently lists the _ibmcloud hpvs_ command you can use to display the status. Copy the entire `ibmcloud hpvs instance crn:...` command above, which has your unique instance id (shown in the output of the above command, it starts with _crn:_), into the code cell below and run it periodically (about once a minute) until the instance is successfully created. 

**Tip:** Your instance is successfully created when the value of the _Cloud state_ field of the output is _active_. For the first several minutes you will see that the _Cloud state_ is _provisioning_ and that most of the values listed below _Cloud state_ are _not available_.  At a certain point, you will see that those values that were _not available_ have been populated, but you may see that the _Cloud state_ remains as _provisioning_ for a little while. Ensure that you wait for the _Cloud state_ to change to _active_ before proceeding with the lab.

In [ ]:
! 
# copy and paste above the entire 'ibmcloud hpvs instance crn:.....' commmand from the \
# output from the previous command, and run it periodically (every minute or so) until it is running.

---

### Step 7.4 Edit this markdown cell to create a clickable link to get to your sample application

Once your instance is active, note the value of the _Public IP address_ field from the output from _Step 7.3_.  

Double-click anywhere within this paragraph to go into _edit_ mode of this markdown cell.  Then, in the final line of this markdown cell, change `xx.xx.xx.xx` to your public IP address.  The actual URL is on the right side of the line (between parentheses). The text of the link is on the left side of the line in between square brackets.  You may as well change both occurrences of `xx.xx.xx.xx` for clarity, but the one that matters is the link on the right side.   (If you don't see both sides you haven't succesfully got into edit mode—keep trying or ask an instructor for help).  After you have made the change, run this markdown cell by clicking the *Run* icon in the Jupyter Notebook menu above. This will render the markdown and present you with the properly formatted link. Click the link and you will bring up your sample secure bitcoin wallet application.

[Change the xx.xx.xx.xx to your public IP and then click here](https://xx.xx.xx.xx/electrum)

---

## Running the Secure Bitcoin Wallet Application in your BYOI HPVS instance

### Overview

This application is a secure [Electrum](https://electrum.org/#home) Bitcoin wallet found at [this GitHub repository](https://github.com/IBM/secure-bitcoin-wallet). You can use it to store the identities necessary to transact on the Bitcoin public blockchain network. While this version of the wallet is just a proof of concept, running this type of wallet on Hyper Protect enables you to protect your digital assets from bad actors such as malicious server administrators.

*This Electrum Bitcoin wallet functions regularly, while inheriting the security benefits of the Hyper Protect Virtual Server platform. This ensures that a legitimate version of the application is running (Secure Image Build process) and that administrators cannot compromise the application (Hyper Protect Virtual Server deployment).*

Here is an overview diagram of the _sample secure bitcoin wallet application_ that you built in this lab: 

![Secure Bitcoin Overview](images/sampleapparch.png)

The application, by default, uses software cryptography in order to generate keys, but it can also be configured to use _Hyper Protect Crypto Services_ in order to generate keys more securely, which is what you have done earlier in this lab when you specified the _ZHSM_ and the _APIKEY_ environment variables in _Step 7.2_.


### Step 8.1 Account Registration

Registering a fake account for lab purposes.

1. Choose `Register` since you won't have an existing account on the newly deployed server.

    ![Secure Bitcoin Wallet Welcome](Bitcoin_Wallet_Images/Bitcoin_Wallet_Homepage.png)

2. Enter the registration details you desire.

    ![User Registration](Bitcoin_Wallet_Images/Wallet_register_info.png)


*Do not worry about the information you enter on the registration page (shown below) as it won't map to anything critical to the real world.  You just need your password to log back in if you log out of the application.*

### Step 8.2 Wallet initialization

It's time to set up your Secure Electrum Bitcoin wallet itself!

1. Bring up the wallet screen

    ![Wallet Start](Bitcoin_Wallet_Images/Click_Wallet_Start.png)

2. Enter in a `password` for your wallet and click `Create Wallet`

    ![Create Wallet](Bitcoin_Wallet_Images/Create_Wallet.png)

3. Load your newly created wallet to enable it

    ![Load Wallet](Bitcoin_Wallet_Images/Load_Wallet.png)

4. Reload the page

    ![Initialized Wallet](Bitcoin_Wallet_Images/Reload_new_wallet.png)

5. Your wallet is ready for use in all of its glory

    ![Locked and Loaded Wallet](Bitcoin_Wallet_Images/Loaded_Wallet_Time.png)

### Step 8.3 Receiving Bitcoins

A wallet without currency is blasphemy, so it's time to fill it on up with fake bitcoins.

1. Go to the `Receive` section of the wallet and get the bitcoin wallet address.  

    ![Wallet receiving bitcoin](Bitcoin_Wallet_Images/Receiving_bitcoin.png)

    *This is the address whose alloted bitcoins you lay claim to.*

2. Open [this bitcoin test faucet site](https://bitcoinfaucet.uo1.net/send.php) [click on the link] and use it to transfer bitcoin value to the `Receiving address` in your wallet **from step 1 above**.

    ![Transfer Bitcoins](Bitcoin_Wallet_Images/Send_bitcoins_to_wallet.png)

    *Instead of transacting on the "real" bitcoin blockchain network, we are using a [testnet](https://medium.com/myetherwallet/understanding-blockchain-changes-testnets-and-mainnets-c2171a8e835f). This means that the network participants agree that the assets on the network have no value. Testnets are used for testing changes to the network, testing applications and introducing new users to the technical aspects of network. There are testnets for many of the public blockchain networks including of course bitcoin and ethereum. Test faucets (like the one we are using) act as distributors of tokens to other users on the testnet.*

3. Switch back to your wallet tab and go to the `History` section and then reload the page.

    ![Check History](Bitcoin_Wallet_Images/Check_History_Receive_Transaction.png)

4. You should now see your wallet address as the recipient of a `pending transaction` from the bitcoin testnet faucet.

    ![First Transaction History](Bitcoin_Wallet_Images/Transaction_History_no_confirmations.png)

5. After clicking for more information you get a detailed transaction overview

    ![Receive Transaction 0 Confirmations](Bitcoin_Wallet_Images/0_confirmations_block_details.png)

    *The number of [confirmations](https://en.bitcoin.it/wiki/Confirmation) a block receives is the number of blocks deep the block that has included the blockchain transaction is. In other words, `1 confirmation` means that the current head block includes the transaction. `2 confirmations` means that the current head block's parent block contains the transaction and so on. The deeper the transaction is (more confirmations it has) the more mining power it would take to "reverse history" and undo the transaction.*

    Since there are `0 confirmations` in the picture above, it means that the transaction is not yet included in a block. This is why it has no timestamp yet and its block number is listed as `0`.

6. After waiting anywhere from a minute to a few minutes (depending on the current testnet traffic) you should see a confirmation on your transaction.

    *You will need to keep reloading the page for it to update. It doesn't update automatically.*

    ![1 Confirmation Received](Bitcoin_Wallet_Images/1_Confirmation_Received.png)

    There is now a timestamp and a block number!

    *As time passes more confirmations will accumulate. You can check on the `confirmations` again at the end of the lab to show yourself this (Given enough time has passed for one or more additional confirmations to occur).*

### Step 8.4 Returning Bitcoins

Since you are feeling charitable you decide to transfer your testnet value back to the faucet's address.

1. Go back to [the faucet site](https://bitcoinfaucet.uo1.net/send.php) and get its bitcoin address

    ![Faucet Address](Bitcoin_Wallet_Images/testnet_faucet_address.png)

    *This address is here for easy copy and paste (ensure this matches what the application shows in the browser before using this, as the bitcoin address they use for returning funds changes occasionally):

    ``` bash
    tb1ql7w62elx9ucw4pj5lgw4l028hmuw80sndtntxt
    ```


2. Send bitcoins to faucet using `Send` section of wallet

    ![Send Bitcoins to Faucet](Bitcoin_Wallet_Images/Send_bitcoins_back_to_faucet.png)

3. Once sent you will see the `Signed transaction` information filled out on your wallet page.

    ![Bitcoins Sent Back to Faucet](Bitcoin_Wallet_Images/bitcoin_send_transaction_output.png)

4. You can go back to the [bitcoin faucet site](https://bitcoinfaucet.uo1.net/send.php) (you may have to refresh the page) and see the transaction as green there with a sender of your address and a transaction ID matching the one you sent.

    ![Bitcoins returned to faucet](Bitcoin_Wallet_Images/transaction_giving_coins_back_to_faucet.png)

5. You can view both the initial received transaction and the newly sent transaction in the `History` tab of your bitcoin wallet.

    *You have to keep reloading the page for it to update with new confirmations. If you wait long enough, eventually these transactions will gain confirmations as in the picture below*

    ![Both transactions with 1 confirmation](Bitcoin_Wallet_Images/both_confirmed_transactions.png)

6. You can continue to play around with the electrum wallet before moving on.

### Summary

You have participated in the bitcoin testnet using a securely built electrum bitcoin wallet powered by a Hyper Protect Virtual Server. Congratulations!!! 

Now, it's time to clean up..

---

## Clean up when you are done with the lab

This section describes important cleanup steps to perform when you have completed the lab. It will guide you through deleting the following artifacts that you created during the lab:

1. The Secure Build Server HPVS instance you created
2. The sample Secure Bitcoin Wallet Application HPVS instance you created
3. The Secure Bitcoin Wallet Application container image that you created for this lab
4. The IBM Container Registry namespace that you created for this lab
5. The IBM Cloud IAM Key you created for this lab
6. The GitHub ssh key you created for this lab
7. The GitHub personal access token you created for this lab


---

### Step 9.1 List your hpvs instances

Run the code cell below to list your HPVS instances:

In [ ]:
! ibmcloud hpvs instances

---

### Step 9.2 Delete your Secure Build Server HPVS instance

In the output from _Step 9.1_, find your instance named *SBContainer*, and copy its CRN (_Cloud Resource Name_). Paste it at the end of the `ibmcloud hpvs instance-delete ` command in the below code cell and then run it.

**Note:** Make sure that your paste operation leaves a space between the end of your instance's CRN and the _#_ character that begins the comment in the code cell. 

In [ ]:
! ibmcloud hpvs instance-delete --force     # <- paste your full CRN name to the left of the pound sign

---

### Step 9.3 Delete your sample Secure Bitcoin Wallet Application HPVS instance

In the output from _Step 9.1_, find your instance named _securewallet_ and copy its CRN value and paste it at the end of the _ibmcloud hpvs instance-delete_ command in the below code cell and then run it.

In [ ]:
! ibmcloud hpvs instance-delete --force    # <- paste your full CRN name to the left of the pound sign

---

### Step 9.4 List your IBM Cloud Container Registry images

Run the following code cell to list your IBM Cloud Container Registry images:

In [ ]:
! ibmcloud cr images

---

### Step 9.5 Delete the container image for the sample secure bitcoin wallet application

From the output from _Step 9.4_, copy and paste the Repository name (first column) of the image you built for this lab, and paste it into the code cell below after `ibmcloud cr image-rm ` and then run the code cell.  This will remove your image from the IBM Cloud Container Registry.

In [ ]:
! ibmcloud cr image-rm    # <- paste your repository name after 'image -rm '

---

### Step 9.6 Verify that your image has been deleted

You can rerun the `ibmcloud cr images` command to verify that you deleted the Secure Bitcoin Wallet Application image. It should not show up in the output from the below code cell:

In [ ]:
! ibmcloud cr images

---

### Step 9.7 List your IBM Cloud Container Registry namespaces

**Note:** If you used an existing namespace and wish to keep it, skip this step and skip _Step 9.8_.

Run the following code cell to list your IBM Cloud Container Registry namespaces:

In [ ]:
! ibmcloud cr namespaces

---

### Step 9.8 Delete the IBM Cloud Container Registry namespace that you created for this lab


Type, or copy and paste from the output from _Step 9.7_, the namespace you created for this lab to the end of the code cell below and then run it.

In [ ]:
! ibmcloud cr namespace-rm --force         # <- paste the name of the namespace you want to delete

---

### Step 9.9 List your IBM Cloud IAM Keys

Run the code cell below to list the IAM Keys associated with your account.

In [ ]:
! ibmcloud iam api-keys

---

### Step 9.10 Delete the IBM Cloud IAM Key you created for this lab

If you used the lab default name of _myapikey_ then the below code cell is ready to run. If you used a different name, find the key that you created for this lab, and copy and paste either the _Name_ (first column) or the ID (last column) into the code cell below, replacing _myapikey_.  Run the code cell in order to delete your IAM key.

In [ ]:
! ibmcloud iam api-key-delete --force  myapikey

---

### Step 9.11 Verify that you deleted the IAM Key you created for the lab

List your API keys again by running the code cell below and ensure that the key you created for this lab is no longer listed in the output:

In [ ]:
! ibmcloud iam api-keys

---

### Step 9.12 Remove the ssh key you created for this lab from your GitHub account

Delete the ssh key you added for the lab from your GitHub account by following steps 1-3 of [these instructions](https://docs.github.com/en/github/authenticating-to-github/reviewing-your-ssh-keys), choosing the _Delete_ button for the ssh key you want to get rid of.  If you were an existing GitHub user prior to this lab, be careful not to delete any other ssh keys you may have added for other purposes.

---

### Step 9.13 Remove the GitHub personal access token you created for this lab from your GitHub account

You should still be logged in to GitHub from the previous step, _Step 9.12_.  Click on your picture or avatar for your account and navigate to _Settings->Developer Settings->Personal access tokens_ and delete the personal access token that you created in _Step 2.5_ of this lab.  Ask an instructor for help if you have difficulty deleting this.

---

## You have reached the end of the lab!

Thank you for taking this lab, and please feel free to provide feedback, either by opening a GitHub issue on the lab repo at https://github.com/ibm-wsc/ibm-cloud-hpvs-sbs-byoi-lab or by sending an email directly to the primary author of the lab, Barry Silliman at "silliman at us dot ibm dot com"  (the email spammers will never figure out my real email address from that, but you will).

---

## Acknowledgements

My esteemed colleagues Jin VanStee and Garrett Woodworth also made significant contributions to this lab, but I take responsibility for any defects or shortcomings in the lab. -  Barry Silliman

## References

This lab is based largely on publicly available material on IBM's website, especially [Tutorial: Using Secure Build Server with a digital wallet](https://cloud.ibm.com/docs/hp-virtual-servers?topic=hp-virtual-servers-tutorial_secure_build_server)

Additional information about using the Secure Build Server CLI can be found [here](https://github.com/ibm-hyper-protect/secure-build-cli)  

Here is the [GitHub repo for the sample secure bitcoin wallet application](https://github.com/IBM/secure-bitcoin-wallet)